In [ ]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc, disvg, wsvg
from svgpathtools import svg2paths, svg2paths2, wsvg
import matplotlib.pyplot as plt
import numpy as np
Path.iscontained = False

seg1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
seg3 = Line(50+100j,400+200j)
#Four Intersections
#seg1 = QuadraticBezier(250+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
#seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
path = Path(seg1, seg2, seg3)

off1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
off2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
off3 = Line(50+100j,400+200j)

off_path = Path(off1,off2,off3)

In [ ]:
# @mesure
def char2paths(char): # Implement character with multiple path, maybe ussing parsing

    plt.figure(figsize=(4, 4))
    plt.text(0.5, 0.5, char[0], fontsize=200, ha='center', va='center')
    plt.axis('off')
    filename='.matplot'
    plt.savefig(filename, format='svg')
    plt.close()
    
    paths, _, _ = svg2paths2(filename)
    paths = paths[1].continuous_subpaths()
    filename='svgtools.svg'
    wsvg(paths, filename=filename)
    print(filename)
    return paths #list

# @mesure
def set_contained_attr(paths: list) -> None:
    for path in paths:
        path.iscontained = any(path.is_contained_by(compare) for compare in paths if compare != path)

In [ ]:
import time
def mesure(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"'{func.__name__}': {execution_time:.6f} seconds.")
        return result
    return wrapper

In [ ]:
# @mesure
def offset_segment(seg,distance,res=5,prev_end=None) -> list: 
    distance = distance * -1
    # print()
    if isinstance(seg,Line):
        offset_vector = distance * seg.normal(0)
        if prev_end and (abs(prev_end - (seg.point(0) + offset_vector))<=10):
            offset_seg = Line(prev_end,seg.point(1) + offset_vector)
        else:
            # offset_vector = distance * seg.normal(0)
            offset_seg = Line(seg.point(0) + offset_vector,seg.point(1) + offset_vector)
        return [offset_seg]

    elif isinstance(seg,(Arc, QuadraticBezier)):
        offset_seg = Path()
        # print('prev:', prev_end,'curr:', seg.point(0) + distance * seg.normal(0))
        
        if prev_end and (abs(prev_end - (seg.point(0) + distance * seg.normal(0)))<=10):
            # print(abs(prev_end - (seg.point(0) + distance * seg.normal(0))))
            curr = prev_end
            # print('ocurre',curr)
        else:
            curr = seg.point(0) + distance * seg.normal(0)
            # print('no ocurre')

        # curr = seg.point(0) + distance * seg.normal(0)    
        curr_tan = seg.unit_tangent(0)
        for k in range(1,res+1):
            t=k / res
            next = seg.point(t) + distance * seg.normal(t)
            next_tan = seg.unit_tangent(t)
            
            # Having curr/next as start/end  of each segment,
            # vectors curr_tan/next_tan pointing to a common point
            # can be used as refference for offset segment as bezier curve
            if curr_tan.real == 0:
                X = curr.real
                Y = (next_tan.imag/next_tan.real) * (X - next.real) + next.imag

            elif next_tan.real == 0:
                X = next.real
                Y = (curr_tan.imag/curr_tan.real) * (X - curr.real) + curr.imag

            else:
                Ax = curr.real; Ay = curr.imag
                Bx = next.real; By = next.imag

                ma = curr_tan.imag/curr_tan.real
                mb = next_tan.imag/next_tan.real

                # solving for lines intersection
                X = ((By-Ay) - (mb * Bx - ma * Ax))/(ma-mb)
                Y = (ma*mb * (Bx-Ax) - (By*ma-Ay*mb))/(mb-ma)

            control = complex(X,Y)
            new_element = QuadraticBezier(curr,control,next)

            # Before appending new_element, check if it intersects with some
            # non consecutive previous element, if so, delete all elements between
            # both of them, chop tails and check for continuity.

            offset_seg.append(new_element)
            # print(curr,next)
            curr = next
            curr_tan = next_tan
        
        return list(offset_seg)  #-> Path

    else:
        raise TypeError("Input `segment` should be a Line, "
                        "QuadraticBezier, CubicBezier, or Arc object.")

def offset_path(path: Path,distance,res=5) -> list:
    result_path = Path()
    # if path.iscontained: distance = distance * -1
    for i, seg in enumerate(path):
        next_seg = path[(i+1)%len(path)]

        # print('Control:',result_path.end)
        if result_path:
            result_path.extend(offset_segment(seg,distance,res,result_path.end)) ###
        else:
            result_path.extend(offset_segment(seg,distance,res)) ###
            pass
        
        # if seg.normal(1) != next_seg.normal(0):
        if abs(seg.normal(1) - next_seg.normal(0))>0.1:

            result_path.append(Arc(start=seg.point(1) - distance * seg.normal(1), ###
                radius=complex(distance, distance),
                rotation=0,
                large_arc=False,
                sweep=distance<0,
                end=next_seg.point(0) - distance * next_seg.normal(0)))
    
    # remove_loops(result_path)
    return result_path


In [ ]:
@mesure
def remove_loops(path:Path) -> Path:
    """Corregir intersecciones en paths abiertos"""    

    # if fail: uncomment i<j part
    nodes = []
    intersections = []
    for i, seg in enumerate(path):
        for ii, other_seg in enumerate(path[i:]):
            j = i+ii
            if j == (i-1)%len(path) or j == i or j == (i+1)%len(path): continue
            if intersection := seg.intersect(other_seg): #invert method, then invert tuple
                # if i<j:
                intersections.append(((i,j),intersection[0]))
            
    #Sort as other_seg
    order = [order[0][1] for order in intersections]
    pairs = zip(order,intersections)
    intersections = [x[1] for x in sorted(pairs, key=lambda x: x[0],reverse=True)]


    #Removing loops
    original_len = len(path)
    for elements in intersections:
        indexes, t_values = elements[0], elements[1]

        #loop on normal path order
        if indexes[1]-indexes[0] <= indexes[0] + original_len - indexes[1]:

            prev = path[indexes[0]].split(t_values[0])[0]
            next = path[indexes[1]].split(t_values[1])[1]
            
            #assert hairless joint and closed loop
            if prev.end == next.start:
                path[indexes[0]:indexes[1]+1] = [prev,next]

            else: 
                intersection = prev.intersect(next)
                i = 0
                while intersection and intersection != [(1.0, 0.0)]: #chop imperfections (hairs)
                    # print('     ')
                    # print('=====')
                    # print('INTERSECTION FAIL (loop):',intersection)
                    # print('=====')
                    # [(1, 0.0)]
                    if i<16:
                        prev = prev.split(intersection[0][0])[0]
                        next = next.split(intersection[0][1])[1]
                    else:

                        prev = prev.split(intersection[0][0])[0]
                        next = next.split(intersection[0][1]+1e-18*i)[1]
                    
                    intersection = prev.intersect(next)
                    i += 1
                # print('LOOP BREAK:',prev.intersect(next))

                intersection = prev.intersect(next)
                if intersection == [(1, 0.0)]:                
                    path[indexes[0]:indexes[1]+1] = [prev,next]

                elif not intersection:                    
                    # print('OPEN CASE:',prev.intersect(next))
                    joint=Line(start=prev.end,end=next.start)
                    # print(joint)
                    path[indexes[0]:indexes[1]+1] = [prev,joint,next]
                    nodes.append(joint.point(0.5))

                else:
                    print('wtf?')
                

        else: #loop jumping from end of path to the beginning 
            prev = path[indexes[1]].split(t_values[1])[0]
            next = path[indexes[0]].split(t_values[0])[1]
            path[indexes[1]:],path[0:indexes[0]+1] = [prev], [next]
            pass

    # print(nodes)
    return nodes

Generating Character SVG

In [ ]:
character = char2paths('}')
outer_wall = []
base = []

set_contained_attr(character)
nodes = []

#Base
for path in character:
    base.append(offset_path(path,200))

#Outer wall
for path in character:
    outer_wall.append(offset_path(path,50))

#Clean Up
for path in outer_wall:
    nodes.extend(remove_loops(path))
    pass

for path in base:
    nodes.extend(remove_loops(path))
    pass

# wsvg(character + offset_character,filename='final.svg',nodes=nodes)
wsvg(character + outer_wall + base,filename='final.svg')
# wsvg(character,nodes=nodes,filename='final.svg')

Generating OBJECT/STL